In [ ]:
from IPython.core.display import Image
from numpy.lib.function_base import append
import numpy as np
from google.colab.patches import cv2_imshow
import cv2 # 導入OpenCV函式庫
from google.colab import files
import matplotlib.pyplot as plt

L = cv2.imread('L.jpg')
R = cv2.imread('R.jpg')
from IPython.core.display import Image
from numpy.lib.function_base import append
import numpy as np
from google.colab.patches import cv2_imshow
import cv2 # 導入OpenCV函式庫
from google.colab import files
import matplotlib.pyplot as plt

L = cv2.imread('L.jpg')
R = cv2.imread('R.jpg')
Lgary = cv2.cvtColor(L, cv2.COLOR_BGR2GRAY)
Rgray = cv2.cvtColor(R, cv2.COLOR_BGR2GRAY)


# ======================================eight points======================================
pts = np.array([[3,244],[225,147],[233,376],[445,214],[14,223],[224,135],[487,211],[234,422]]) #left points
ptsp = np.array([[60,251],[278,151],[242,377],[489,210],[74,231],[279,138],[535,205],[236,421]]) #right pointsp
for i in range(0, len(pts)):
      # print(corners[i,0])
      # print(corners[i,1])
      cv2.circle(R, (int(ptsp[i,0]), int(ptsp[i,1])), 4, (0,255,0), 0)
      cv2.circle(L, (int(pts[i,0]), int(pts[i,1])), 4, (0,255,0), 0)
# cv2_imshow(L)
# cv2_imshow(R)
# =========================================================================================

F, mask = cv2.findFundamentalMat(pts, ptsp, cv2.FM_8POINT)# find Fundamental matrix

# =====================draw epipolar lines===============================================
def drawlines(img1,img2,lines,pts1,pts2):
  r,c = img1.shape
  img1 = cv2.cvtColor(img1,cv2.COLOR_GRAY2BGR)
  img2 = cv2.cvtColor(img2,cv2.COLOR_GRAY2BGR)
  for r,pt1,pt2 in zip(lines,pts1,pts2):
    color = tuple(np.random.randint(0,255,3).tolist())
    x0,y0 = map(int, [0, -r[2]/r[1] ])
    x1,y1 = map(int, [c, -(r[2]+r[0]*c)/r[1] ])
    img1 = cv2.line(img1, (x0,y0), (x1,y1), color,1)
    img1 = cv2.circle(img1,tuple(pt1),5,color,-1)
    img2 = cv2.circle(img2,tuple(pt2),5,color,-1)
  return img1,img2
# ========================================================================================

# Find epilines corresponding to points in right image (second image) and
# drawing its lines on left image
lines1 = cv2.computeCorrespondEpilines(ptsp.reshape(-1,1,2), 2,F)
lines1 = lines1.reshape(-1,3)
img5,img6 = drawlines(Lgary,Rgray,lines1,pts,ptsp)
# ========================================================================================

# Find epilines corresponding to points in left image (first image) and
# drawing its lines on right image
lines2 = cv2.computeCorrespondEpilines(pts.reshape(-1,1,2), 1,F)
lines2 = lines2.reshape(-1,3)
img3,img4 = drawlines(Rgray,Lgary,lines2,ptsp,pts)
cv2_imshow(img5)
cv2_imshow(img3)
plt.subplot(121),plt.imshow(img5)
plt.subplot(122),plt.imshow(img3)
plt.show()
# ========================================================================================

print('fundamental matrix:\n',F)


# =====================estimate error===============================================
dis_err = np.zeros(len(pts))
print('L distance value:')

for i in range(0, len(pts)):
  dis = abs((lines1[i,0]*pts[i,0]+lines1[i,1]*pts[i,1]+lines1[i,2]))/np.sqrt(lines1[i,0]**2+lines1[i,1]**2)
  dis_err[i] = dis
  print('point',i+1,'error:',dis)
  
print('R distance value:')
for i in range(0, len(ptsp)):
  dis = abs((lines2[i,0]*ptsp[i,0]+lines2[i,1]*ptsp[i,1]+lines2[i,2]))/np.sqrt(lines2[i,0]**2+lines2[i,1]**2)
  dis_err[i] = dis
  print('point',i+1,'error:',dis)
